In [1]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
# Own Libraries
from utility.utils import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

# CATALOG CREATION

In [2]:
df = pd.read_csv('DATA/lunar_crater_database_robbins_2018.csv')
df = df[['LAT_CIRC_IMG','LON_CIRC_IMG','DIAM_CIRC_IMG']]
df.rename( columns= {'LAT_CIRC_IMG':'lat', 'LON_CIRC_IMG':'lon','DIAM_CIRC_IMG':'r'}, inplace=True )
df.r = df.r/2
df.lon = np.where(df.lon < 180, df.lon, df.lon - 360)
df = df.sort_values(by='lat')
df.insert(3,'i',range(len(df)))
df = df.sort_values(by='lon')
df.insert(4,'j',range(len(df)))
df = df[df.r < 100]
df.reset_index(drop=True, inplace=True)

In [3]:
# init H
H = pd.DataFrame({
'Lat_min':[0],
'Lat_max':[0],
'Lon_min':[0],
'Lon_max':[0],
'ICD_max':[0],
'angle1':[0],
'angle2':[0],
'angle3':[0],
'crt_name1':[0],
'crt_name2':[0],
'crt_name3':[0]
})

In [4]:
df.shape[0]

1296735

In [6]:
for ind in range(200000,300000):
    triplet = find_triplet(df, ind)
    K_v = compute_K_vet(triplet)
    ICD_max = find_max_crt_dist(triplet)
    lat_m, lat_M, lon_m, lon_M = find_bounds(triplet)
    R_M = find_max_radius(triplet)
    entry = [lat_m, lat_M, lon_m, lon_M, ICD_max, K_v[0], K_v[1], K_v[2], triplet[0].name,triplet[1].name,triplet[2].name]
    if np.all(K_v > 15):
        H.loc[ind] = entry
    printProgressBar(ind, df.shape[0], printEnd='')    
    # SAVE EACH 1.000 entries:
    if ind % 1000 == 0 & ind !=0:
        file_name = f'TRN_catalog_part_2.csv'
        H.to_csv(file_name)
        break
    print(ind)

 |███████████████-------------------------------------------------------------------------------------| 15.4% 

NameError: name 'shift' is not defined

# Matching: 

In [ ]:
# def matching(crat_det, crat_cat):
    
#     mat1 = crat_det
#     mat2 = crat_cat

#     MATCHER = []
#     for i in range(mat1.shape[0]):
#         for j in range(mat2.shape[0]):
#             f1 = mat1[i, :]
#             f2 = mat2[j, :]
#             tmp = sq_dif(f1,f2)
#             TMP = [i, j, tmp]
#             MATCHER.append(TMP)
#     flag = np.zeros(2)
#     for elem in MATCHER:
#         if elem[2] < 100:
#             match = [elem[0],elem[1]] 
#             flag = np.vstack((flag, match))
#     flag = flag[1:,:]
#     flag = np.array(flag).astype(int)
#     return flag


In [ ]:
# flag = np.zeros(2)
# for elem in MATCHER:
#     if elem[2] < 100:
#         match = [elem[0],elem[1]] 
#         flag = np.vstack((flag, match))
# flag = flag[1:,:]
# flag = np.array(flag).astype(int)
# flag

In [ ]:
# # Stack the images
# img1 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_1_Lat_0_Long_-147.8129.jpg')
# img2 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_2_Lat_0_Long_-147.7069.jpg')
# img_matched = np.hstack((img1,img2)).astype(int)
# plt.imshow(img_matched, cmap='gray')
# plt.axis(False)
# plt.grid(False)

In [ ]:
# cv2.arrowedLine(img_matched,start,end,(255, 0, 0) , 3)

In [ ]:
# center_coordinates = start
# radius = 500
# color = (0,0,255)
# thickness = 6
# cv2.circle(img_matched, center_coordinates, radius, color, thickness)